In [216]:
import numpy as np
import pandas as pd
%matplotlib inline 
import re
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
plt.rcParams['figure.figsize'] = (8, 6)
import xlsxwriter
import scipy.stats as stats

In [217]:
#name of experiment
name = 'Multirun_SuggestUsersForChat_run1'
# files with payments of all users
tg = pd.read_csv(r'C:\Users\user\Downloads\report-1524485834.csv')
cg1 = pd.read_csv(r'C:\Users\user\Downloads\report-1524485847.csv')
cg2 = pd.read_csv(r'C:\Users\user\Downloads\report-1524485856.csv')
tg['group'] = 'tg'
cg1['group'] = 'cg1'
cg2['group'] = 'cg2'
df= tg.append(cg1.append(cg2))

In [218]:
df.head()

,user_id,sex,isReferral,isPaid,active_days_0_1day,active_days_1day_2day,active_days_2day_3day,like_sent_0_1day,like_sent_1day_2day,like_sent_2day_3day,...,fist_dialog_received_fake_0_1day,fist_dialog_received_fake_1day_2day,fist_dialog_received_fake_2day_3day,first_dialog_started_fake_0_1day,first_dialog_started_fake_1day_2day,first_dialog_started_fake_2day_3day,payment_0_1day,payment_1day_2day,payment_2day_3day,group
0,117978263,1,0,0,1,0,1,6,0,0,...,0,0,0,0,0,0,0,0,0,tg
1,118028416,1,0,1,1,0,0,7,0,0,...,0,0,0,0,0,0,1,0,0,tg
2,118003862,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,tg
3,118009783,1,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,tg
4,117991988,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tg


In [219]:
writer = pd.ExcelWriter('results_of_'+name+'.xlsx', engine='openpyxl') 

In [220]:
j = 0

In [221]:
def minus_fake_column(variable, period):
    df[variable+'-fake'+period] = df[variable+period] - df['fake_'+variable+period]


In [222]:
def mean_and_part(value, period):
        a = pd.DataFrame(df.groupby('group')[value + period].mean()).transpose()
        b = pd.DataFrame(df[df[value + period] >0].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df.groupby('group')['user_id'].count()).transpose()
        return a.append(b)

In [223]:
def mean_and_part2(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        c = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value+ '-fake' + period].mean()).transpose()
        d = pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value+ '-fake' + period] >0)].groupby('group')['user_id'].count().transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b.append(c.append(d)))

In [224]:
def mean_and_part3(value, period):
        a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
        b =(pd.DataFrame(df[(df['active_days' + period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/
            pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
        return a.append(b)

In [225]:
def mean_median_nonzero(value, period):
        a = pd.DataFrame(df.groupby('group')[value + period].mean()).transpose()
        b = pd.DataFrame(df.groupby('group')[value + period].median()).transpose()
        c = pd.DataFrame(df[df[value + period] >0].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df.groupby('group')['user_id'].count()).transpose()
        return pd.concat([a,b,c], axis = 0)

In [226]:
def mean_median_nonzero2(value, period):
    a = pd.DataFrame(df.groupby('group')[value + period].mean()).transpose()
    b = pd.DataFrame(df.groupby('group')[value + period].median()).transpose()
    c =pd.DataFrame(df[df[value + period] >0].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df.groupby('group')['user_id'].count()).transpose()
    d = pd.DataFrame(df.groupby('group')[value+ '-fake' + period].mean()).transpose()
    e = pd.DataFrame(df.groupby('group')[value+ '-fake' + period].median()).transpose()
    f = pd.DataFrame(df[df[value+ '-fake' + period] >0].groupby('group')['user_id'].count().transpose()/
    pd.DataFrame(df.groupby('group')['user_id'].count()).transpose())
    return pd.concat([a,b,c,d,e,f], axis = 0)

In [227]:
def mean_median_nonzero3(value, period):
    a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
    b = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].median()).transpose()
    c =(pd.DataFrame(df[(df['active_days' + period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
    return pd.concat([a,b,c], axis = 0)

In [228]:
def mean_median_nonzero4(value, period):
    a = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].mean()).transpose()
    b = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value + period].median()).transpose()
    c =(pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value + period] >0)].groupby('group')['user_id'].count()).transpose()/pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
    d = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value+ '-fake' + period].mean()).transpose()
    e = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')[value+ '-fake' + period].median()).transpose()
    f = pd.DataFrame(df[(df['active_days'+ period] >0) & (df[value+ '-fake' + period] >0)].groupby('group')['user_id'].count().transpose()/pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose())
    return pd.concat([a,b,c,d,e,f], axis = 0)

In [229]:
def summary_statistics(period):
    b = mean_median_nonzero('mutual', period)
    c = mean_median_nonzero('like_sent', period)
    d = mean_median_nonzero('like_received', period)
    e = mean_median_nonzero('message_sent', period)
    f = mean_median_nonzero('message_received', period)
    g = mean_median_nonzero('first_dialog_started', period)
    h = mean_median_nonzero('fist_dialog_received', period)
    k = pd.DataFrame(df.groupby('group')['user_id'].count()).transpose()
    
    m = pd.concat([b,c,d,e,f,g,h,k],axis = 0, join = 'inner')
    m['dif'] = m['tg']/(m['cg1']+m['cg2'])*2-1
    cols = ['tg', 'cg1', 'cg2', 'dif']
    m = m[cols]
    m.columns = ['TG', 'CG1', 'CG2', 'dif']
    m.index = ['mutual_mean','mutual_median', 'mutual_nonzero_part', 
               'like_sent_mean','like_sent_median', 'like_sent_nonzero_part', 
               'like_received_mean','like_received_median','like_received_nonzero_part', 
               'message_sent_mean','message_sent_median', 'message_sent_nonzero_part',
                'message_received_mean', 'message_received_median', 'message_received_nonzero_part',
                'first_dialog_started_mean','first_dialog_started_median', 'first_dialog_started_nonzero_part',
                'fist_dialog_received_mean','fist_dialog_received_median', 'fist_dialog_received_nonzero_part', 'users_count']
    cm = sns.light_palette("yellow", as_cmap=True)
    return (m.fillna(0))

In [230]:
def summary_statistics2(period):
    b = mean_median_nonzero3('mutual', period)
    c = mean_median_nonzero3('like_sent', period)
    d = mean_median_nonzero3('like_received', period)
    e = mean_median_nonzero3('message_sent', period)
    f = mean_median_nonzero3('message_received', period)
    g = mean_median_nonzero3('first_dialog_started', period)
    h = mean_median_nonzero3('fist_dialog_received', period)
    k = pd.DataFrame(df[df['active_days' + period] >0].groupby('group')['user_id'].count()).transpose()
    
    m = pd.concat([b,c,d,e,f,g,h,k],axis = 0, join = 'inner')
    m['dif'] = m['tg']/(m['cg1']+m['cg2'])*2-1
    cols = ['tg', 'cg1', 'cg2', 'dif']
    m = m[cols]
    m.columns = ['TG', 'CG1', 'CG2', 'dif']
    m.index = ['mutual_mean','mutual_median', 'mutual_nonzero_part', 
               'like_sent_mean','like_sent_median', 'like_sent_nonzero_part', 
               'like_received_mean','like_received_median','like_received_nonzero_part', 
               'message_sent_mean','message_sent_median', 'message_sent_nonzero_part',
                'message_received_mean', 'message_received_median', 'message_received_nonzero_part',
                'first_dialog_started_mean','first_dialog_started_median', 'first_dialog_started_nonzero_part',
                'fist_dialog_received_mean','fist_dialog_received_median', 'fist_dialog_received_nonzero_part', 'users_count']
    cm = sns.light_palette("yellow", as_cmap=True)
    return (m.fillna(0))

In [231]:
def make_table(summary_statistics, norm):
    m1 = summary_statistics('_0_1day')
    m2 = summary_statistics('_1day_2day')
    m3 = summary_statistics('_2day_3day')

    m = pd.concat([m1,m2,m3], axis = 1)
    m.columns = ['TG_1'+ norm,  'CG1_1'+ norm,  'CG2_1'+ norm,  'dif_1'+ norm, 
                 'TG_2'+ norm,  'CG1_2'+ norm,  'CG2_2'+ norm,  'dif_2'+ norm, 
                 'TG_3'+ norm,  'CG1_3'+ norm,  'CG2_3'+ norm,  'dif_3'+ norm]
    cm = sns.light_palette("yellow", as_cmap=True)
    
    k=1
    global j
    k+= j        
    globals()['j'] = j + 4 + m.count(axis = 0)[0]
    
    m.fillna(0).style.format({'dif_1'+ norm: '{:.2%}', 'dif_2'+ norm: '{:.2%}',
                    'dif_3'+ norm: '{:.2%}'}).background_gradient(subset=['dif_1'+ norm, 'dif_2'+ norm, 'dif_3'+ norm], cmap=cm).to_excel(writer, "Main", startrow = k)

In [232]:
minus_fake_column('like_sent', '_0_1day')
minus_fake_column('like_sent', '_1day_2day')
minus_fake_column('like_sent', '_2day_3day')

minus_fake_column('mutual', '_0_1day')
minus_fake_column('mutual', '_1day_2day')
minus_fake_column('mutual', '_2day_3day')

In [233]:
make_table(summary_statistics, '')
make_table(summary_statistics2, 'active')

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\pandas\io\formats\style.py:883: RuntimeWarning: invalid value encountered in double_scalars
  norm = colors.Normalize(s.min() - (rng * low),
c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\pandas\io\formats\style.py:884: RuntimeWarning: invalid value encountered in double_scalars
  s.max() + (rng * high))
c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\matplotlib\colors.py:489: RuntimeWarning: invalid value encountered in less
  np.copyto(xa, -1, where=xa < 0.0)


In [234]:
writer.save()

In [236]:
m.fillna(0).style.format({'dif_1': '{:.2%}', 'dif_2': '{:.2%}',
                    'dif_3': '{:.2%}'}).background_gradient(subset=['dif_1', 'dif_2', 'dif_3'], cmap=cm).set_caption('All users')

,TG_1,CG1_1,CG2_1,dif_1,TG_2,CG1_2,CG2_2,dif_2,TG_3,CG1_3,CG2_3,dif_3
mutual,4.14935,4.09036,3.86889,4.27%,6.62575,9.80078,10.9029,-35.99%,5.47603,8.9797,5.74237,-25.61%
mutual_nonzero_part,0.269325,0.258883,0.294536,-2.67%,0.4284,0.4375,0.465409,-5.11%,0.38843,0.436548,0.417317,-9.02%
mutual-fake,3.51766,3.39594,3.11088,8.12%,5.7136,8.48047,9.71094,-37.18%,4.66777,7.40609,4.68311,-22.78%
mutual-fake_nonzero_part,0.195873,0.179695,0.215407,-0.85%,0.367028,0.394531,0.392956,-6.79%,0.345455,0.365482,0.361604,-4.98%
like_sent,45.8195,37.9218,45.0265,10.48%,56.9832,87.4453,58.4931,-21.91%,32.7983,55.4315,33.0933,-25.90%
like_sent_nonzero_part,0.485834,0.481218,0.518805,-2.84%,0.454874,0.445312,0.451321,1.46%,0.42314,0.482234,0.398864,-3.95%
like_sent-fake,40.0063,29.4376,38.6492,17.52%,52.9218,77.3477,53.6111,-19.18%,31.6182,52.5584,31.3605,-24.65%
like_sent-fake_nonzero_part,0.440714,0.429442,0.475682,-2.62%,0.411552,0.390625,0.403774,3.61%,0.403306,0.451777,0.376863,-2.66%
like_received,16.518,16.7374,15.9068,1.20%,9.75454,9.56803,11.1335,-5.76%,6.06772,5.29971,6.75125,0.70%
like_received_nonzero_part,0.906508,0.923882,0.862365,1.50%,0.3667,0.357755,0.346082,4.20%,0.307235,0.294957,0.293275,4.46%
